# Use AutoAI and AI lifecycle to predict credit risk with `ibm-watsonx-ai`

This notebook contains the steps and code to demonstrate support of AI Lifecycle features of am AutoAI model in watsonx.ai. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository. It also introduces commands for training, persisting and deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Work with watsonx.ai experiments to train AutoAI models.
-  Persist an external model in watsonx.ai repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.
-  Update previously persisted model.
-  Redeploy model in-place.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Optimizer definition](#definition)
3. [Experiment Run](#run)
4. [Persist and Deploy trained AutoAI model](#persistence)
5. [Refine and persist new version of the model](#refine)
6. [Redeploy and score new version of the model](#redeploy)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, contact with your Cloud Pak for Data administrator and ask for your account credentials.

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
%pip install -U wget | tail -n 1
%pip install -U lale | tail -n 1
%pip install -U autoai-libs | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

Enter your watsonx.ai password and hit enter:  ········


#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="definition"></a>
## 2. Optimizer definition

### Training data connection

Define connection information to training data CSV file. This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/data/credit_risk/credit_risk_training_light.csv).

In [7]:
filename = "credit_risk_training_light.csv"

Download training data from git repository.

In [8]:
import os, wget

url = "https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cpd5.2/data/credit_risk/credit_risk_training_light.csv"
if not os.path.isfile(filename):
    wget.download(url)

In [9]:
asset_details = client.data_assets.create("credit_risk_training", filename)
asset_details

Creating data asset...
SUCCESS


{'metadata': {'space_id': '9c4eb218-96e8-41e0-8cd5-f783504a11ea',
  'usage': {'last_updated_at': '2025-05-22T11:39:54Z',
   'last_updater_id': '1000331001',
   'last_update_time': 1747913994246,
   'last_accessed_at': '2025-05-22T11:39:54Z',
   'last_access_time': 1747913994246,
   'last_accessor_id': '1000331001',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'1000331001': {'user_iam_id': '1000331001',
     'roles': ['OWNER']}}},
  'is_linked_with_sub_container': False,
  'name': 'credit_risk_training',
  'description': '',
  'asset_type': 'data_asset',
  'origin_country': 'us',
  'resource_key': 'credit_risk_training',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': '32c875f2-f104-4801-9d5f-a0987098790c',
  'created': 1747913994246,
  'created_at': '2025-05-22T11:39:54Z',
  'owner_id': '1000331001',
  'size': 0,
  'version': 2.0,
  'asset_state': 'available',
  'asset_attributes': ['data_asset'],
  'asset_id': 'd414512e-a854-49c3-a8

In [10]:
from ibm_watsonx_ai.helpers import DataConnection, AssetLocation


credit_risk_conn = DataConnection(
    location=AssetLocation(asset_id=client.data_assets.get_id(asset_details))
)

training_data_reference = [credit_risk_conn]

### Optimizer configuration

Provide the input information for AutoAI optimizer:
- `name` - experiment name
- `prediction_type` - type of the problem
- `prediction_column` - target column name
- `scoring` - optimization metric

In [11]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, space_id=space_id)

pipeline_optimizer = experiment.optimizer(
    name="Credit Risk Prediction - AutoAI",
    prediction_type=AutoAI.PredictionType.BINARY,
    prediction_column="Risk",
    scoring=AutoAI.Metrics.ROC_AUC_SCORE,
)

Configuration parameters can be retrieved via `get_params()`.

In [12]:
pipeline_optimizer.get_params()

{'name': 'Credit Risk Prediction - AutoAI',
 'desc': '',
 'prediction_type': 'binary',
 'prediction_column': 'Risk',
 'prediction_columns': None,
 'timestamp_column_name': None,
 'scoring': 'roc_auc',
 'holdout_size': None,
 'max_num_daub_ensembles': None,
 't_shirt_size': 'm',
 'train_sample_rows_test_size': None,
 'include_only_estimators': None,
 'include_batched_ensemble_estimators': None,
 'backtest_num': None,
 'lookback_window': None,
 'forecast_window': None,
 'backtest_gap_length': None,
 'cognito_transform_names': None,
 'csv_separator': ',',
 'excel_sheet': None,
 'encoding': 'utf-8',
 'positive_label': None,
 'drop_duplicates': True,
 'outliers_columns': None,
 'text_processing': None,
 'word2vec_feature_number': None,
 'daub_give_priority_to_runtime': None,
 'text_columns_names': None,
 'sampling_type': None,
 'sample_size_limit': None,
 'sample_rows_limit': None,
 'sample_percentage_limit': None,
 'number_of_batch_rows': None,
 'n_parallel_data_connections': None,
 'test_

<a id="run"></a>
## 3. Experiment run

Call the `fit()` method to trigger the AutoAI experiment. You can either use interactive mode (synchronous job) or background mode (asychronous job) by specifying `background_model=True`.

In [13]:
run_details = pipeline_optimizer.fit(
    training_data_reference=training_data_reference, background_mode=False
)

Training job f2a01a01-0f16-4e0d-9748-895e7b795c4a completed: 100%|████████| [01:11<00:00,  1.39it/s]


You can use the `get_run_status()` method to monitor AutoAI jobs in background mode.

In [14]:
pipeline_optimizer.get_run_status()

'completed'

#### Load trained model

In [15]:
pipeline = pipeline_optimizer.get_pipeline()

#### Reading training data from COS

In [16]:
train_df = pipeline_optimizer.get_data_connections()[0].read()

train_X = train_df.drop(["Risk"], axis=1).values
train_y = train_df.Risk.values

<a id="persistence"></a>
## 4. Persist and Deploy trained AutoAI model

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

### Online deployment creation

In [17]:
from ibm_watsonx_ai.deployment import WebService

service = WebService(credentials, source_space_id=space_id)

service.create(
    experiment_run_id=run_details["metadata"]["id"],
    model="Pipeline_1",
    deployment_name="AutoAI credit-risk deployment",
)

Preparing an AutoAI Deployment...
Published model uid: b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed
Deploying model b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed using V4 client.


######################################################################################

Synchronous deployment creation for id: 'b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.....
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='433799b3-5756-4563-96f1-603ab2f5ff1e'
-----------------------------------------------------------------------------------------------




Deployment object could be printed to show basic information:

In [ ]:
print(service)

To show all available information about the deployment use the `.get_params()` method:

In [ ]:
service.get_params()

### Scoring of webservice
You can make scoring request by calling `score()` on deployed pipeline.

In [20]:
predictions = service.score(payload=train_df.drop(["Risk"], axis=1).iloc[:10])
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.815472082175866, 0.18452791782413402]],
    ['No Risk', [0.7501849542379176, 0.24981504576208238]],
    ['Risk', [0.44323551396938976, 0.5567644860306102]],
    ['No Risk', [0.7860837153311935, 0.2139162846688065]],
    ['Risk', [0.10388810002019544, 0.8961118999798046]],
    ['Risk', [0.05423329461659787, 0.9457667053834021]],
    ['Risk', [0.39761734701550944, 0.6023826529844906]],
    ['No Risk', [0.7195913804536169, 0.2804086195463831]],
    ['No Risk', [0.8252891188726867, 0.1747108811273132]],
    ['Risk', [0.05226738963105093, 0.9477326103689491]]]}]}

If you want to work with the web service in an external Python application you can retrieve the service object by:
 - Initialize the service by `service = WebService(wml_credentials)`
 - Get deployment_id by `service.list()` method
 - Get webservice object by `service.get('deployment_id')` method

After that you can call `service.score()` method.

### Deleting deployment
You can delete the existing deployment by calling the `service.delete()` command.
To list the existing web services you can use `service.list()`.

<a id="refine"></a>
## 5. Refine and persist new version of the model

In this section, you'll learn how to store new refinement version of your model in watsonx.ai repository.

### Pipeline refinement with [Lale](https://github.com/IBM/lale)

In this section you learn how to refine and retrain the best
pipeline returned by AutoAI. There are many ways to refine a pipeline.
For illustration, simply replace the final estimator in the
pipeline by an interpretable model. The call to
`wrap_imported_operators()` augments scikit-learn operators with
schemas for hyperparameter tuning.

In [21]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier as Tree
from sklearn.neighbors import KNeighborsClassifier as KNN
from lale.lib.lale import Hyperopt
from lale import wrap_imported_operators

wrap_imported_operators()

#### Pipeline decomposition and new definition

Start by removing the last step of the pipeline, i.e., the final estimator.

In [22]:
prefix = pipeline.remove_last().freeze_trainable()
prefix.export_to_sklearn_pipeline()

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140367139217696',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'float_int_n...
                                                                 ('numpyreplacemissingvalues',
                                                                  NumpyReplaceMissingValues(missing_values=[])),
                                                                 ('numimputer',
                                                                  NumImputer(missing_values=nan,
                                                                             strategy='median')),
                                                                 ('optstandardscaler',
                                                                  OptStandardScaler(use_scaler_flag=False)),
                                                                 ('float32_transform',
                                                                  float32_transform())]))])),
                ('numpypermutearray',
                 NumpyPermuteArray(axis=0,
                                   permutation_indices=[0, 2, 3, 5, 6, 7, 8, 9,
                                                        10, 11, 13, 14, 15, 16,
                                                        17, 18, 19, 1, 4,
                                                        12]))])

Next, add a new final step, which consists of a choice of three
estimators. In this code, `|` is the or combinator (algorithmic
choice). It defines a search space for another optimizer run.

In [23]:
new_pipeline = prefix >> (LR | Tree | KNN)

#### New optimizer `Hyperopt` configuration and training

To automatically select the algorithm and tune its hyperparameters, we
create an instance of the `Hyperopt` optimizer and `fit` it to the
data.

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_X, train_y, test_size=0.15, random_state=33
)

In [25]:
hyperopt = Hyperopt(estimator=new_pipeline, cv=3, max_evals=20, scoring="roc_auc")
hyperopt_pipelines = hyperopt.fit(X_train, y_train)

100%|██████████| 20/20 [00:08<00:00,  2.34trial/s, best loss: -0.7994345630215195]
1 out of 20 trials failed, call summary() for details.
Run with verbose=True to see per-trial exceptions.


In [26]:
pipeline_model = hyperopt_pipelines.get_pipeline(astype="sklearn")

#### Save newly refined pipeline

In [27]:
import joblib

model_path = "refined_pipeline.pickle"
joblib.dump(pipeline_model, model_path)

['refined_pipeline.pickle']

In [28]:
import tarfile

with tarfile.open(model_path + ".tar.gz", "w:gz") as tar:
    tar.add(model_path)
model_path += ".tar.gz"

### Publish new version of the model

#### Download current version of the model

In [29]:
model_id = service.get_params()["entity"]["asset"]["id"]
deployment_id = service.get_params()["metadata"]["id"]

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


In [30]:
downloaded_model_name = "current_model.zip"
client.repository.download(model_id, downloaded_model_name)

Successfully saved model content to file: 'current_model.zip'


'/home/wsuser/work/current_model.zip'

In [31]:
import zipfile

with zipfile.ZipFile(downloaded_model_name, "r") as zip_ref:
    zip_ref.extractall()

#### Create archive containing updated model

In [32]:
updated_model_path = "updated_model.zip"
with zipfile.ZipFile(updated_model_path, "w") as zip_ref:
    zip_ref.write(model_path)
    zip_ref.write("pipeline-model.json")

#### Save the current model version

In [33]:
import json

print(json.dumps(client.repository.create_model_revision(model_id), indent=2))

{
  "metadata": {
    "name": "P1",
    "space_id": "9c4eb218-96e8-41e0-8cd5-f783504a11ea",
    "resource_key": "81444767-8506-4af1-9fd3-9b2b2dd4b3a6",
    "id": "b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed",
    "created_at": "2025-05-22T11:41:53Z",
    "rev": "1",
    "commit_info": {
      "committed_at": "2025-05-22T11:44:01Z"
    },
    "rov": {
      "member_roles": {
        "1000331001": {
          "user_iam_id": "1000331001",
          "roles": [
            "OWNER"
          ]
        }
      }
    },
    "owner": "1000331001"
  },
  "entity": {
    "pipeline": {
      "id": "91cd7ec2-6bda-417e-82d1-ff8368b03952"
    },
    "software_spec": {
      "id": "8c1a58c6-62b5-4dc4-987a-df751c2756b6",
      "name": "hybrid_0.1"
    },
    "type": "wml-hybrid_0.1",
    "training_data_references": [
      {
        "type": "data_asset",
        "connection": null,
        "location": {
          "href": "/v2/assets/d414512e-a854-49c3-a8fc-cceff65dd884?space_id=9c4eb218-96e8-41e0-8cd5-f783504

#### Update the model

In [34]:
metadata = {client.repository.ModelMetaNames.NAME: "AutoAI credit-risk updated model"}

published_model = client.repository.update_model(
    model_id=model_id, update_model=updated_model_path, updated_meta_props=metadata
)

#### Save new model revision of the updated model

In [35]:
new_model_revision = client.repository.create_model_revision(model_id)
print(json.dumps(new_model_revision, indent=2))

{
  "metadata": {
    "name": "AutoAI credit-risk updated model",
    "space_id": "9c4eb218-96e8-41e0-8cd5-f783504a11ea",
    "resource_key": "73dd6496-5468-4080-ab0e-2f50aeb7cbe6",
    "id": "b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed",
    "created_at": "2025-05-22T11:41:53Z",
    "rev": "2",
    "commit_info": {
      "committed_at": "2025-05-22T11:44:04Z"
    },
    "rov": {
      "member_roles": {
        "1000331001": {
          "user_iam_id": "1000331001",
          "roles": [
            "OWNER"
          ]
        }
      }
    },
    "owner": "1000331001"
  },
  "entity": {
    "pipeline": {
      "id": "91cd7ec2-6bda-417e-82d1-ff8368b03952"
    },
    "software_spec": {
      "id": "8c1a58c6-62b5-4dc4-987a-df751c2756b6",
      "name": "hybrid_0.1"
    },
    "type": "wml-hybrid_0.1",
    "training_data_references": [
      {
        "type": "data_asset",
        "connection": null,
        "location": {
          "href": "/v2/assets/d414512e-a854-49c3-a8fc-cceff65dd884?space_id=9

**Note:** Model revisions can be identified by model `id` and `rev` number. 

Get model `rev` number from creation details:

In [36]:
rev_id = new_model_revision["metadata"].get("rev")

#### List list existing revisions of the model

In [37]:
client.repository.list_models_revisions(model_id)

,REV,NAME,CREATED
0,2,AutoAI credit-risk updated model,2025-05-22T11:41:53Z
1,1,P1,2025-05-22T11:41:53Z


#### Get details of updated model

In [38]:
print(json.dumps(client.repository.get_details(model_id), indent=2))

{
  "metadata": {
    "name": "AutoAI credit-risk updated model",
    "space_id": "9c4eb218-96e8-41e0-8cd5-f783504a11ea",
    "resource_key": "73dd6496-5468-4080-ab0e-2f50aeb7cbe6",
    "id": "b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed",
    "created_at": "2025-05-22T11:41:53Z",
    "commit_info": {
      "committed_at": "2025-05-22T11:41:53Z"
    },
    "rov": {
      "member_roles": {
        "1000331001": {
          "user_iam_id": "1000331001",
          "roles": [
            "OWNER"
          ]
        }
      }
    },
    "owner": "1000331001"
  },
  "entity": {
    "pipeline": {
      "id": "91cd7ec2-6bda-417e-82d1-ff8368b03952"
    },
    "software_spec": {
      "id": "8c1a58c6-62b5-4dc4-987a-df751c2756b6",
      "name": "hybrid_0.1"
    },
    "type": "wml-hybrid_0.1",
    "training_data_references": [
      {
        "type": "data_asset",
        "connection": null,
        "location": {
          "href": "/v2/assets/d414512e-a854-49c3-a8fc-cceff65dd884?space_id=9c4eb218-96e8-41e

<a id="redeploy"></a>
## 6. Redeploy and score new version of the model

In this section, you'll learn how to redeploy new version of the model by using the watsonx.ai Client.

In [39]:
metadata = {
    client.deployments.ConfigurationMetaNames.ASSET: {"id": model_id, "rev": rev_id}
}

updated_deployment = client.deployments.update(deployment_id, changes=metadata)

Since ASSET is patched, deployment need to be restarted. 


########################################################################

Deployment update for id: '433799b3-5756-4563-96f1-603ab2f5ff1e' started

########################################################################


updating........
ready


---------------------------------------------------------------------------------------------
Successfully finished deployment update, deployment_id='433799b3-5756-4563-96f1-603ab2f5ff1e'
---------------------------------------------------------------------------------------------




Wait for the deployment update: 

In [40]:
import time

status = None
while status not in ["ready", "failed"]:
    print(".", end=" ")
    time.sleep(2)
    deployment_details = client.deployments.get_details(deployment_id)
    status = deployment_details["entity"]["status"].get("state")

print("\nDeployment update finished with status: ", status)

. Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

Deployment update finished with status:  ready


#### Get updated deployment details

In [41]:
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
{
  "entity": {
    "asset": {
      "id": "b78e56d5-6a90-477b-8ab8-b58d5e5ce1ed",
      "rev": "2"
    },
    "chat_enabled": false,
    "custom": {},
    "deployed_asset_type": "model",
    "hybrid_pipeline_hardware_specs": [
      {
        "hardware_spec": {
          "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
          "name": "S",
          "num_nodes": 1
        },
        "node_runtime_id": "auto_ai.kb"
      }
    ],
    "name": "AutoAI credit-risk deployment",
    "online": {},
    "space_id": "9c4eb218-96e8-41e0-8cd5-f783504a11ea",
    "status": {
      "inference": [
        {
          "url": "https://cpd-cpd-instance.apps.ocpwmlautoai-x86-2-nonfips.cp.fyre.ibm.com/ml/v4/deployments/433799b3-5756-4563-96f1-603ab2f5ff1e/predictions"
        }
      ],
      "message": {
        "level": "warning",
        "text": "Successfully patched the asset."
      },
      "online_url":

### Score updated model

In [42]:
scoring_payload = {"input_data": [{"values": train_df.drop(["Risk"], axis=1).iloc[:5]}]}

Use `client.deployments.score()` method to run scoring.

In [43]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [44]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          "No Risk",
          [
            0.815472082175866,
            0.18452791782413402
          ]
        ],
        [
          "No Risk",
          [
            0.7501849542379176,
            0.24981504576208238
          ]
        ],
        [
          "Risk",
          [
            0.44323551396938976,
            0.5567644860306102
          ]
        ],
        [
          "No Risk",
          [
            0.7860837153311935,
            0.2139162846688065
          ]
        ],
        [
          "Risk",
          [
            0.10388810002019544,
            0.8961118999798046
          ]
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps

You successfully completed this notebook!  

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Jan Sołtysik**, Intern at watsonx.ai

**Rafał Chrzanowski**, Software Engineer Intern at watsonx.ai.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.